# Optimizing Python code with Cython

Load cython extension:

In [14]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Python Gauss solver:

Below is the linear equation solver using gaussian elimination we implemented back in week 2.

It does the following checks:
- If dimensions of input numpy arrays are valid
- Equation is unsolvable (non-unique solution)
- It performs pivotting, to avoid zero division

In [15]:
import numpy as np

#Defining a funtion to perform Back Substitution 
def backSubs(P, q):
    unknowns = len(q)
    x = np.zeros(unknowns, dtype = complex)             # Solution vector x of size n
    x[unknowns-1] = q[unknowns-1]/P[unknowns-1,unknowns-1]    # Find x[unknowns-1]
    for i in range(unknowns-2,-1,-1):      # Loop from the end to the beginning
        sum = 0 #Inititalise sum variable
        for j in range(i+1, unknowns):        # For known x values, sum and move to rhs
            sum = sum + P[i][j]*x[j]
        x[i] = (q[i] - sum)/P[i][i]
    return x


def solver(A, b):
    # Check if input is valid
    unknowns = len(b) #Find number of unknown variables
    for i in range(len(A)):
        count = 0
        for j in range(len(A[i])):
            count+=1
        if count != unknowns:
            raise Exception ("Invalid Dimensions") #Raise exception if dimensions are invalid

    #Triangularization of matrix
    for i in range(unknowns):
        max = abs(A[i][i]) 
        row_max = i #Row containg element with maximum absolute value
        for j in range(i + 1, unknowns):
            if max < abs(A[j][i]): 
                max = abs(A[j][i])
                row_max = j

        if max == 0: #Check if all elements of column are zero, if yes raise exception
            raise Exception("Matrix A is singular")
        if row_max != i: #Interchange row if 0 is present
            A[[i, row_max]] = A[[row_max, i]]
            b[[i, row_max]] = b[[row_max, i]]

        for j in range(i + 1, unknowns): #Loop through rows to triangularize matrix A
            norm = A[j][i] / A[i][i] 
            for m in range(i, unknowns):
                A[j][m] = A[j][m] - norm * A[i][m] #Applying row operations
            b[j] = b[j] - norm * b[i] 

    return backSubs(A, b) #Use backsubstitution to return vector x after matrix A is triangularized

## Import Cython

In [16]:
%%cython --annotate
import numpy as np

# Defining a function to perform Back Substitution 
def unop_backSubs(P, q):
    unknowns = len(q)
    x = np.zeros(unknowns, dtype = complex)             # Solution vector x of size n
    x[unknowns-1] = q[unknowns-1]/P[unknowns-1,unknowns-1]    # Find x[unknowns-1]
    for i in range(unknowns-2,-1,-1):      # Loop from the end to the beginning
        sum = 0 #Inititalise sum variable
        for j in range(i+1, unknowns):        # For known x values, sum and move to rhs
            sum = sum + P[i][j]*x[j]
        x[i] = (q[i] - sum)/P[i][i]
    return x


def unop_solver(A, b):
    # Check if input is valid
    unknowns = len(b) #Find number of unknown variables
    for i in range(len(A)):
        count = 0
        for j in range(len(A[i])):
            count+=1
        if count != unknowns:
            raise Exception ("Invalid Dimensions") #Raise exception if dimensions are invalid

    #Triangularization of matrix
    for i in range(unknowns):
        max = abs(A[i][i]) 
        row_max = i #Row containg element with maximum absolute value
        for j in range(i + 1, unknowns):
            if max < abs(A[j][i]): 
                max = abs(A[j][i])
                row_max = j

        if max == 0: #Check if all elements of column are zero, if yes raise exception
            raise Exception("Matrix A is singular")
        if row_max != i: #Interchange row if 0 is present
            A[[i, row_max]] = A[[row_max, i]]
            b[[i, row_max]] = b[[row_max, i]]

        for j in range(i + 1, unknowns): #Loop through rows to triangularize matrix A
            norm = A[j][i] / A[i][i] 
            for m in range(i, unknowns):
                A[j][m] = A[j][m] - norm * A[i][m] #Applying row operations
            b[j] = b[j] - norm * b[i] 

    return unop_backSubs(A, b) #Use backsubstitution to return vector x after matrix A is triangularized

The yellow highlights suggest python interactions. We wish to make optimize the code using cython.

## Optimizing code with Cython

In [4]:
%%cython --annotate
import numpy as np
import cython
cimport numpy as np
cimport cython

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)

cdef c_backSubs(P, q):
    cdef int unknowns, i, j

    unknowns = len(q)
    i = unknowns-2
    x = np.zeros(unknowns, dtype = complex)             # Solution vector x of size n
    
    x[unknowns-1] = q[unknowns-1]/P[unknowns-1,unknowns-1]    # Find x[unknowns-1]
    while i>-1:
        sum = 0 #Inititalise sum variable
        j=i+1
        # For known x values, sum and move to rhs
        while j < unknowns:
            sum = sum + P[i][j]*x[j]
            j+=1
        x[i] = (q[i] - sum)/P[i][i]
        i-=1
    return x

def c_solver(A, b):
    #Check if input is valid
    cdef int i
    i = 0
    cdef Py_ssize_t unknowns=b.shape[0]
    cdef Py_ssize_t len_A = A.shape[0]
    
    while(i<len(A)):
        if len(A[i]) != unknowns:
            raise Exception ("Invalid Dimensions") #Raise exception if dimensions are invalid
        i+=1
    
    cdef int j, p, row_max, n, m
    p = 0
    cdef float max

    while(p<unknowns):
        max = abs(A[p][p]) 
        row_max = p #Row containg element with maximum absolute value
        j = p+1
        while j<unknowns:
            if max < abs(A[j][p]): 
                max = abs(A[j][p])
                row_max = j
            j+=1
        
        if max == 0.: #Check if all elements of column are zero, if yes raise exception
            raise Exception("Matrix A is singular")
        
        if row_max != p: #Interchange row if 0 is present
            A[[p, row_max]] = A[[row_max, p]]
            b[[p, row_max]] = b[[row_max, p]]
        
        n = p + 1
        while n < unknowns:
            norm = A[n][p] / A[p][p] 
            m = p
            while m < unknowns:
                A[n][m] = A[n][m] - norm * A[p][m] #Applying row operations
                m+=1
            b[n] = b[n] - norm * b[p] 
            n+=1
        p+=1
    
    return c_backSubs(A, b) #Use backsubstitution to return vector x after matrix A is triangularized


We can see from the generated `HTML`, it is mostly white with a few highlights of yellow. This suggest more C interaction which will improve its speed.

The following optimizations were done:
- Using `cdef` to initialize variables with pre-defined datatypes. Python is dynamic language, and variable datatypes can be rewritten. While this gives flexibility, this needs a check of the data type everytime it is called. </br>
Hence, by using `cdef` we constrain its datatype.  

    - `int` for counting variables
    - `str` for string variables
    - `complex` for matrix elements, etc.
    
- `@cython.cdivision(True)`: This is done to make the division faster. This is usually risky as error checks suck as zero division isn't done. Here, we perform pivoting so we will not encouter zero division.
- `@cython.boundscheck(False)`: This again makes our code faster, but it neglects checking for bounds, which is acceptable as we have coded it such that it doesn't go out of bounds.
- `@cython.wraparound(False)`: C doesn't include negative indices. Setting it to `True` would constrain the code further but at the cost of slowing down.
- `range()` is a python implementation to increment in steps and it can't be implemented in C. Hence, we use pure C loops (in our case - `while`) in place of `range` in `for` loops.


## Test our optimization

Let's test our optimizations in a typic 10x10 randomly generated matrix.

In [17]:
# Randomly generated 10x10 matrix
A = np.random.rand(10,10)
B = np.random.rand(10,1)
print(solver(A, B))
print(unop_solver(A, B))
print(c_solver(A, B))

%timeit solver(A,B)
%timeit unop_solver(A,B)
%timeit c_solver(A,B)

[-0.5623704 +0.j -0.19136851+0.j  0.34056336-0.j -0.61558872-0.j
  1.01430853-0.j -0.07825736+0.j  0.49648885+0.j  0.49598212+0.j
 -0.03546915-0.j -0.1075738 +0.j]
[-0.5623704 +0.j -0.19136851+0.j  0.34056336-0.j -0.61558872-0.j
  1.01430853-0.j -0.07825736+0.j  0.49648885+0.j  0.49598212+0.j
 -0.03546915-0.j -0.1075738 +0.j]
[-0.5623704 +0.j -0.19136851+0.j  0.34056336-0.j -0.61558872-0.j
  1.01430853-0.j -0.07825736+0.j  0.49648885+0.j  0.49598212+0.j
 -0.03546915-0.j -0.1075738 +0.j]
330 µs ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
304 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
229 µs ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Time taken: Cython < Unoptimized < Python

The optimized cython code has improved the time taken by python by over 30%.

# Optimizing the spice simulator

Unoptimized Python code (with cython import):

In [18]:
%%cython --annotate
import numpy as np
from sys import exit
import math

# Defining a function to perform Back Substitution 
def unop_backSubs(P, q):
    unknowns = len(q)
    x = np.zeros(unknowns, dtype = complex)             # Solution vector x of size n
    x[unknowns-1] = q[unknowns-1]/P[unknowns-1,unknowns-1]    # Find x[unknowns-1]
    for i in range(unknowns-2,-1,-1):      # Loop from the end to the beginning
        sum = 0 #Inititalise sum variable
        for j in range(i+1, unknowns):        # For known x values, sum and move to rhs
            sum = sum + P[i][j]*x[j]
        x[i] = (q[i] - sum)/P[i][i]
    return x


def unop_solver(A, b):
    # Check if input is valid
    unknowns = len(b) #Find number of unknown variables
    for i in range(len(A)):
        count = 0
        for j in range(len(A[i])):
            count+=1
        if count != unknowns:
            raise Exception ("Invalid Dimensions") #Raise exception if dimensions are invalid

    #Triangularization of matrix
    for i in range(unknowns):
        max = abs(A[i][i]) 
        row_max = i #Row containg element with maximum absolute value
        for j in range(i + 1, unknowns):
            if max < abs(A[j][i]): 
                max = abs(A[j][i])
                row_max = j

        if max == 0: #Check if all elements of column are zero, if yes raise exception
            raise Exception("Matrix A is singular")
        if row_max != i: #Interchange row if 0 is present
            A[[i, row_max]] = A[[row_max, i]]
            b[[i, row_max]] = b[[row_max, i]]

        for j in range(i + 1, unknowns): #Loop through rows to triangularize matrix A
            norm = A[j][i] / A[i][i] 
            for m in range(i, unknowns):
                A[j][m] = A[j][m] - norm * A[i][m] #Applying row operations
            b[j] = b[j] - norm * b[i] 

    return unop_backSubs(A, b) #Use backsubstitution to return vector x after matrix A is triangularized

# Read netlist
def readfile(file):
    try: 
        with open(file, "r") as filehandle:
            data = filehandle.read().split("\n")
        # Raise exception if file not found
    except Exception as ex:
        print(ex)
        return

    freq = None
    for line in data:
        if line == '':
            continue
        i = line.split()
        # Remove garbage lines
        if i[0] == '.circuit':   start = data.index(line)
        if i[0] == '.end':  end = data.index(line)
        if i[0] == '.ac':
            if freq is not None:
                if freq!= i[2]:
                    print("Different frequencies found!")
                    exit()
            else:
                freq = float(i[2])
    if freq is None:
        freq = 0.

    for line in data:
        if line == '':
            continue
        i = line.split()
        # Raise exception if both AC and DC source is provided
        if (i[0][0] == 'V' or i[0][0] == 'I') and freq != 0 and i[3] == 'dc':
            print("Different frequencies found!")
            exit()
    if start > end:
        raise Exception("Invalid circuit definition")
    
    return data[start+1: end], freq

# Generate MNA matrix
def matrixGenerator(file):
    lines, f = readfile(file)
    nodes_str = [] # List storing nodes
    nodes_dict = {} # List mapping nodes to variable 'count' 
    count = 0
    for line in lines:
        if line == '':
            continue
        i = line.split()
        for j in range(1, 3):
            if i[j] not in nodes_str:
                nodes_str.append(i[j])
                nodes_dict[i[j]] = count
                count+=1

    # curr_dict stores a mapping of Voltage Source and corresponding nodes and current variable                
    curr_dict = {}
    curr_count = count
    for line in lines:
        if line == '':
            continue
        i = line.split()
        if i[0][0] == 'V':
            curr_dict[i[0]] = (i[1], i[2], curr_count)
            curr_count+=1
    # Assign matrix of size curr_count
    matB = [0] * curr_count
    matA = []
    for i in range(curr_count):
        matA.append([])
        for j in range(curr_count):
            matA[i].append(0)

    for j in nodes_dict.keys():
        # Writing KCL at every node (j)
        for line in lines:
            if line == '':
                continue
            i = line.split()
            # Iterate across lines looking for node j
            for k in range(1, 3):
                if i[k] == j:
                    if i[0][0] != 'V' and i[0][0] != 'I':
                        # Compute impedance for R, L, C
                        if i[0][0] == 'R':
                            imp = float(i[3])
                        elif i[0][0] == 'C':
                            imp = ((0.0-1.0j)*(1/(float(i[3])*f*2*np.pi)))
                        elif i[0][0] == 'L':
                            imp = (float(i[3])*f*2*np.pi*(0.0+1.0j))
                        
                        #Increment at corresponding element in matrix
                        if(k == 1):             
                            matA[nodes_dict[j]][nodes_dict[i[1]]] += 1/imp
                            matA[nodes_dict[j]][nodes_dict[i[2]]] -= 1/imp
                            
                        elif(k == 2):
                            matA[nodes_dict[j]][nodes_dict[i[2]]] += 1/imp
                            matA[nodes_dict[j]][nodes_dict[i[1]]] -= 1/imp
                        
                    elif i[0][0] == 'I':
                        if(k == 1):
                            matB[nodes_dict[j]] = -float(i[4])
                        elif(k == 2):
                            matB[nodes_dict[j]] = float(i[4])
                    
                    # Finding map of Voltage source with corresponding current source
                    elif i[0][0] == 'V':
                        if(k == 1):
                            matA[nodes_dict[j]][curr_dict[i[0]][2]] = 1
                        elif(k == 2):
                            matA[nodes_dict[j]][curr_dict[i[0]][2]] = -1
            
            # Adding Voltage source equation to matrix
            if i[0][0] == 'V':
                matA[curr_dict[i[0]][2]][nodes_dict[i[1]]] = 1
                matA[curr_dict[i[0]][2]][nodes_dict[i[2]]] = -1
                if f == 0:
                    matB[curr_dict[i[0]][2]] = float(i[4])
                else:
                    matB[curr_dict[i[0]][2]] = float(i[4]) * (math.cos(float(i[5])) + 1j * math.sin(float(i[5])))
    return matA, matB, nodes_dict, curr_dict

# Function to log output onto console
def Output_logger(file):
    matA, matB, nodes_dict, curr_dict = matrixGenerator(file)
    ref = nodes_dict['GND'] # Finding reference potential (GND)
    matA1 = list(matA)
    matB1 = list(matB)
    # Deleting reference nodal voltage in matrix
    for i in range(len(matA1)):
        del matA1[i][ref]    
    del matA1[ref]
    del matB1[ref]
    
    # Finding solution vector using solver()
    sol = unop_solver(np.array(matA1, dtype = complex), np.array(matB1, dtype = complex))
    del nodes_dict['GND']

    # Printing output Node voltages and auxillary currents
# Uncomment the following for spice solver output
########################################################

    # print("Ref. Potential: GND") 
    # for i in nodes_dict.keys():
    #     print(f"Voltage at {i} = {sol[nodes_dict[i]-1]:.10f}")
    # for i in curr_dict.keys():
    #     print("The current through",i,"from node",curr_dict[i][0],"to",curr_dict[i][1],f"= {sol[curr_dict[i][2]-1]:.10f}")

########################################################

    del matA, matA1, nodes_dict, matB, matB1, curr_dict, sol


The yellow highlights on the generated `HTML` show python interaction. Again, applying the same optimizations as before along with:

- To avoid repeated lookups in an array, we store it in a variable and perform operations with it.

Applying the similar optimizations in our spice simulator:

In [7]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython
from cpython.list cimport PyList_GET_ITEM, PyList_GET_SIZE

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)


cdef c_backSubs(P, q):
    cdef int unknowns, i, j

    unknowns = len(q)
    i = unknowns-2
    x = np.zeros(unknowns, dtype = complex)             # Solution vector x of size n
    
    x[unknowns-1] = q[unknowns-1]/P[unknowns-1,unknowns-1]    # Find x[unknowns-1]
    while i>-1:
        sum = 0 #Inititalise sum variable
        j=i+1
        # For known x values, sum and move to rhs
        while j < unknowns:
            sum = sum + P[i][j]*x[j]
            j+=1
        x[i] = (q[i] - sum)/P[i][i]
        i-=1
    return x

def c_solver(A, b):
    #Check if input is valid
    cdef int i
    i = 0
    cdef Py_ssize_t unknowns=b.shape[0]
    cdef Py_ssize_t len_A = A.shape[0]
    
    while(i<len(A)):
        if len(A[i]) != unknowns:
            raise Exception ("Invalid Dimensions") #Raise exception if dimensions are invalid
        i+=1
    
    cdef int j, p, row_max, n, m
    p = 0
    cdef float max

    while(p<unknowns):
        max = abs(A[p][p]) 
        row_max = p #Row containg element with maximum absolute value
        j = p+1
        while j<unknowns:
            if max < abs(A[j][p]): 
                max = abs(A[j][p])
                row_max = j
            j+=1
        
        if max == 0.: #Check if all elements of column are zero, if yes raise exception
            raise Exception("Matrix A is singular")
        
        if row_max != p: #Interchange row if 0 is present
            A[[p, row_max]] = A[[row_max, p]]
            b[[p, row_max]] = b[[row_max, p]]
        
        n = p + 1
        while n < unknowns:
            norm = A[n][p] / A[p][p] 
            m = p
            while m < unknowns:
                A[n][m] = A[n][m] - norm * A[p][m] #Applying row operations
                m+=1
            b[n] = b[n] - norm * b[p] 
            n+=1
        p+=1
    
    return c_backSubs(A, b) #Use backsubstitution to return vector x after matrix A is triangularized


from sys import exit
import math
# Read netlist

def c_readfile(str file):
    try: 
        with open(file, "r") as filehandle:
            data = filehandle.read().split("\n")
        # Raise exception if file not found
    except Exception as ex:
        print(ex)
        return

    cdef int start, end
    cdef float freq=-1.
    cdef str first
    for line in data:
        if line == '':
            continue
        sep = line.split()
        first = sep[0]
        # Remove garbage lines
        if first == '.circuit':   start = data.index(line)
        if first == '.end':  end = data.index(line)
        if first == '.ac':
            if freq is not -1.:
                if freq!= sep[2]:
                    print("Different frequencies found!")
                    exit()
            else:
                freq = float(sep[2])
    if freq is -1.:
        freq = 0.

    for line in data:
        if line == '':
            continue
        i = line.split()
        # Raise exception if both AC and DC source is provided
        if (i[0][0] == 'V' or i[0][0] == 'I') and freq != 0 and i[3] == 'dc':
            print("Different frequencies found!")
            exit()
    if start > end:
        raise Exception("Invalid circuit definition")
    
    return data[start+1: end], freq

# Generate MNA matrix
def c_matrixGenerator(str file):
    cdef float f
    cdef list lines
    lines, f = c_readfile(file)
    cdef str line
    cdef list nodes_str
    cdef dict nodes_dict
    cdef int count =  0
    nodes_str = [] # List storing nodes
    nodes_dict = {} # List mapping nodes to variable 'count' 

    for line in lines:
        if line == '':
            continue
        i = line.split()
        for j in range(1, 3):
            if i[j] not in nodes_str:
                nodes_str.append(i[j])
                nodes_dict[i[j]] = count
                count+=1

    # curr_dict stores a mapping of Voltage Source and corresponding nodes and current variable                
    cdef dict curr_dict
    cdef int curr_count
    curr_dict = {}
    curr_count = count
    
    for line in lines:
        if line == '':
            continue
        i = line.split()
        if i[0][0] == 'V':
            curr_dict[i[0]] = (i[1], i[2], curr_count)
            curr_count+=1
            
    # Assign matrix of size curr_count
    cdef list matA, matB
    matB = [0] * curr_count
    matA = []


    for i in range(curr_count):
        matA.append([])
        for j in range(curr_count):
            matA[i].append(0)

    cdef int k
    cdef str compo
    cdef complex imp
    for j in nodes_dict.keys():
        # Writing KCL at every node (j)
        for line in lines:
            if line == '':
                continue
            i = line.split()
        
            k = 1
            while k < 3:
                compo = i[0][0]
                if i[k] == j:
                    if compo != 'V' and compo != 'I':
                        # Compute impedance for R, L, C
                        if compo == 'R':
                            imp = float(i[3])
                        elif compo == 'C':
                            imp = ((0.0-1.0j)*(1/(float(i[3])*f*2*np.pi)))
                        elif compo == 'L':
                            imp = (float(i[3])*f*2*np.pi*(0.0+1.0j))
                        
                        #Increment at corresponding element in matrix
                        if(k == 1):             
                            matA[nodes_dict[j]][nodes_dict[i[1]]] += 1/imp
                            matA[nodes_dict[j]][nodes_dict[i[2]]] -= 1/imp
                            
                        else:
                            matA[nodes_dict[j]][nodes_dict[i[2]]] += 1/imp
                            matA[nodes_dict[j]][nodes_dict[i[1]]] -= 1/imp
                        
                    elif compo == 'I':
                        if(k == 1):
                            matB[nodes_dict[j]] = -float(i[4])
                        else:
                            matB[nodes_dict[j]] = float(i[4])
                    
                    # Finding map of Voltage source with corresponding current source
                    elif compo == 'V':
                        if(k == 1):
                            matA[nodes_dict[j]][curr_dict[i[0]][2]] = 1
                        else:
                            matA[nodes_dict[j]][curr_dict[i[0]][2]] = -1
                k+=1
            # Adding Voltage source equation to matrix
            if compo == 'V':
                matA[curr_dict[i[0]][2]][nodes_dict[i[1]]] = 1
                matA[curr_dict[i[0]][2]][nodes_dict[i[2]]] = -1
                if f == 0:
                    matB[curr_dict[i[0]][2]] = float(i[4])
                else:
                    matB[curr_dict[i[0]][2]] = float(i[4]) * (math.cos(float(i[5])) + 1j * math.sin(float(i[5])))
    return matA, matB, nodes_dict, curr_dict

# Function to log output onto console
def c_Output_logger(str file):
    cdef list matA, matB, matA1, matB1
    cdef dict nodes_dict, curr_dict

    matA, matB, nodes_dict, curr_dict = c_matrixGenerator(file)

    cdef int ref, j = 0
    ref = nodes_dict['GND'] # Finding reference potential (GND)
    matA1 = list(matA)
    matB1 = list(matB)
    # Deleting reference nodal voltage in matrix
    
    while j < PyList_GET_SIZE(matA1):
        del matA1[j][ref]    
        j+=1
    del matA1[ref]
    del matB1[ref]
    
    # Finding solution vector using solver()
    sol = c_solver(np.array(matA1, dtype = complex), np.array(matB1, dtype = complex))
    del nodes_dict['GND']


    # Printing output Node voltages and auxillary currents
# Uncomment the following for spice solver output

########################################################
    # cdef str i
    # print("Ref. Potential: GND") 
    # for i in nodes_dict.keys():
    #     print(f"Voltage at {i} = {sol[nodes_dict[i]-1]:.10f}")
    # for i in curr_dict.keys():
    #     print("The current through",i,"from node",curr_dict[i][0],"to",curr_dict[i][1],f"= {sol[curr_dict[i][2]-1]:.10f}")

########################################################

    del matA, matA1, nodes_dict, matB, matB1, curr_dict, sol


## Comparing times

In [12]:
filename = 'ckt3.netlist'
print('Unoptimized python code:')
%timeit Output_logger(filename)

Unoptimized python code:
315 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
print('\nOpimized python code with cython')
%timeit Output_logger(filename)


Opimized python code with cython
173 µs ± 3.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
